In [1]:
import plotly.graph_objects as go
# import kaleido
from plotly.subplots import make_subplots
import plotly.io as pio
import pandas as pd
import numpy as np
import json

from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm


import sys
sys.path.insert(0, "../scripts")
import helpers as h
import seaborn as sns

In [ ]:
# general code 

# get time series (ts)
# do smoothing function (ts_smooth)
# subtract (ts - ts_smooth)
# derivative d/dt(ts - ts_smooth), and second deriv d/dt(ts - ts_smooth)
# look at distributions, calculate standard deviations
# calculate accuracy of guesses 

In [2]:
def take_derivative(series):
    return pd.Series(h.normalize(np.gradient(series)))

In [3]:
def plot_distribution(deriv, deriv2, marker_width=0.1, bin_size=0.003):
    fig = go.Figure()

    fig.add_trace(go.Histogram(
        x=deriv2, histnorm='probability', name='2nd Derivative', opacity=0.9, marker_line=dict(width=marker_width ,color='black'), xbins=dict( size=bin_size),))
    
    fig.add_trace(go.Histogram(
        x=deriv, histnorm='probability', name='1st Derivative', opacity=1, marker_line=dict(width=marker_width ,color='black'), xbins=dict( size=bin_size),))

    fig.update_layout(barmode="stack")

    return fig

In [51]:
class Window_Detect2:
    def __init__(self, df):
        self.time = df["DateTime"]
        self.window = df["Window Open"]
        self.window_norm = h.normalize(self.window)
        self.temp = df["Temp C"]
        self.temp_norm = h.normalize(self.temp)

    def analyze_window_change(self, smooth_fx):
        self.smooth_series = h.normalize(smooth_fx(self.temp_norm))
        # TODO: mean largest - mean smallest?
        self.dif = h.normalize(self.temp_norm - self.smooth_series)
        self.deriv = take_derivative(self.dif)
        self.deriv2 = take_derivative(self.deriv)
        self.std, self.std2 = self.deriv.std(), self.deriv2.std()

    def plot_analysis(self):
        fig = go.Figure()

        series = [self.window_norm, self.temp_norm, self.smooth_series, self.dif, self.deriv, self.deriv2]
        names = ["Window", "Observed", "Smoothed", "Difference", "Deriv1", "Deriv2"]

        for name, ser in zip(names, series):
            fig.add_trace(go.Scatter(x=self.time, y=ser, name=name, mode='lines'))

        return fig

    def plot_distributions(self, marker_width=0.1, bin_size=0.003):
        print(f"Std 1 = {self.std}, Std 2 = {self.std2}")
        fig = go.Figure()

        for ix, ser in enumerate([self.deriv2, self.deriv]):
            opacity = 0.9 if ix == 0 else 1
            fig.add_trace(go.Histogram(
            x=ser, histnorm='probability', name=f' Derivative {2 - ix}', opacity=opacity, marker_line=dict(width=marker_width ,color='black'), xbins=dict( size=bin_size),))

        fig.update_layout(barmode="stack")

        return fig



In [46]:
a00, a01 = h.import_desired_data("A", "15T")
print(a00["Window Open"].unique(), a01["Window Open"].unique(), a00.columns)

[1.] [0. 1.] Index(['DateTime', 'Temp C', 'RH %', 'Room', 'Ambient Temp', 'Ambient RH',
       'Window Open'],
      dtype='object')


In [52]:
w1 = Window_Detect2(a01)
w1.analyze_window_change(lambda x: x+0.01,)
w1.plot_distributions()

Std 1 = 0.07224500135781263, Std 2 = 0.07988020552985585


In [32]:
w1.plot_analysis()

In [12]:
w1.smooth_series

0      0.037569
1      0.027898
2      0.026782
3      0.091133
4      0.142837
         ...   
380    0.007439
381    0.006695
382    0.000000
383    0.000000
384    0.002976
Name: Temp C, Length: 385, dtype: float64

In [ ]:
def detect_window_change(series, window_series, smooth_fx):
    norm_series = h.normalize(series)
    smooth_series = h.normalize(smooth_fx(series))
    # TODO: mean largest - mean smallest?
    dif = h.normalize(series - smooth_series)
    deriv = take_derivative(dif)
    deriv2 = take_derivative(deriv)
    std, std2 = deriv.std, deriv2.std


In [ ]:
# stl technique use in simulation => derivs and guesses, maybe note standard deviation? 

In [ ]:
# sinusoid smoothing technique

In [ ]:
# exponentially weighted mean 